In [ ]:
%load_ext autoreload
%autoreload 2
import os

from vespa.deployment import VespaCloud
from vespa.package import (
    ApplicationPackage,
    ContainerCluster,
    ContentCluster,
    DeploymentConfiguration,
    Document,
    Field,
    FieldSet,
    Nodes,
    Schema,
)

package = ApplicationPackage(
    name="testapp",
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(name="text", type="string", indexing=["summary"]),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["text"])],
        )
    ],
)
package.clusters = [
    ContentCluster(
        id="doc_content",
        nodes=Nodes(count="2"),
        document_name="doc",
        min_redundancy="2",
    ),
    ContainerCluster(
        id="doc_container",
        nodes=Nodes(count="2"),
    ),
]
package.deployment_config = DeploymentConfiguration(
    environment="prod", regions=["aws-us-east-1c"]
)

In [ ]:
vespa_cloud = VespaCloud(
    tenant=os.environ["TENANT_NAME"],
    application="testapp",
    application_package=package,
)

In [ ]:
vespa_cloud.deploy_to_prod()

In [ ]:
import time

instance_name = "default"
build_no = vespa_cloud.deploy_to_prod()
# Wait until buildstatus is succeeded
max_wait = 600  # 10 minutes
start = time.time()
while time.time() - start < max_wait:
    build_status = vespa_cloud.check_production_build_status(build_no=build_no)
    if (
        build_status[f"production-{vespa_cloud.get_prod_region()}"]["status"]
        == "success"
    ):
        print("Deployment succeeded")
        break
        app = vespa_cloud.get_application()

In [ ]:
app = vespa_cloud.get_application()

In [ ]:
status = app.get_application_status()